In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os
import json

In [2]:
response = requests.get('http://store.steampowered.com/appreviews/240760?json=1&start_offset=0')

In [3]:
response

<Response [200]>

In [4]:
response.json()

{'success': 1,
 'query_summary': {'num_reviews': 3,
  'review_score': 8,
  'review_score_desc': 'Very Positive',
  'total_positive': 2468,
  'total_negative': 559,
  'total_reviews': 3027},
 'reviews': [{'recommendationid': '93179222',
   'author': {'steamid': '76561197991991243',
    'num_games_owned': 281,
    'num_reviews': 2,
    'playtime_forever': 3632,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 3632,
    'last_played': 1445238679},
   'language': 'english',
   'review': 'The mechanics of the game are way too visible to the player and it ruins immersion.   Basically every map is:  if you go left you need 7 lockpick skill to pass, go right for a 6 hack check, go straight for traps that need level 4 skill to disarm.  It\'s fine to have skill checks like this but they should be hidden behind the scenes and masked by the story and atmosphere.  You aren\'t going left because it has a lockpick 7 option, you are going left to help the help the princess.  You are going r

# Michael notes

the above does work in retrieving reviews. cannot find a way to pull top games by genre from api, referring to old scraping data from previouw notebook, going to scrape list of games, then getting their game ids, and using those id's in the api, at least to start working on nlp examples. 

In [5]:
response2 = requests.get('https://store.steampowered.com/tags/en/RPG/#p=0&tab=TopRated')

In [6]:
response2

<Response [200]>

In [16]:
page = response2.text

In [22]:
# modifed code to find the top 15 RPG games

soup = BeautifulSoup(page, 'html5lib')
table = soup.find('div', id='TopRatedRows')
table
rows = [row for row in table.find_all('a')]
rows[0]
len(rows)

15

In [28]:
'''
code created for previous project, pulled for reference
'''


# games = {}

# for row in rows:
#     title = (row.find('span', class_='title').text)
#     url = row.get('href')
#     release_date = (row.find('div', class_='col search_released responsive_secondrow').text)
#     if (row.find('div', class_='col search_discount responsive_secondrow').text.strip()) == '':
#         been_sale = 0
#     else:
#         been_sale = 1
#     if been_sale == 0:
#         orig_price = (row.find('div', class_='col search_price responsive_secondrow').text.strip())
#     else:
#         orig_price = (row.find('strike').text)
#     games[title] = (url, 
#                     release_date,
#                     been_sale,
#                     orig_price)

AttributeError: 'NoneType' object has no attribute 'text'

In [33]:
#new code for title of games

rows[0].find(class_='tab_item_name').text

'The Witcher® 3: Wild Hunt'

In [64]:
# find app id - could only find in url without major digging, luckily multiple game urls have been formatted the same. 

# url = rows[0].get('href')
# url.split('/')[4]

# rows[1].get('href').split('/')[4]


app_id = rows[0].get('href').split('/')[4]
app_id

'292030'

In [61]:
rows[1].get('href').split('/')[4]

'413150'

In [65]:
games = {}

for row in rows:
    title = (row.find(class_='tab_item_name').text)
    app_id = row.get('href').split('/')[4]
    
    games[title] = (app_id)

In [67]:
top_15_rpg_list = games

In [75]:
rpg_ids = list(top_15_rpg.values())

In [76]:
rpg_ids

['292030',
 '413150',
 '22380',
 '1145360',
 '49520',
 '391540',
 '48700',
 '435150',
 '374320',
 '213670',
 '582010',
 '22330',
 '489830',
 '261550',
 '1235140']

# ok
now that I have app ids for the top 15 rpg games, lets pull the game info from api and pull 20 comments per game

In [78]:
## manually checking first id for viability before creating loop.

response = requests.get('http://store.steampowered.com/appreviews/292030?json=1')

In [81]:
witcher3 = response.json()

In [96]:
# so the api produces review information in a dictionary>list>dictionary.

witcher3['reviews'][1]['review']

'In my top 3 games of all time.\nAlso in an effort to save some unfortunate soul from my mistake, I must warn you....if you mod this game and get the Fast Travel from anywhere mod, you can completely break this game and lock yourself out from being able to complete main quests.\nDO NOT USE THE FAST TRAVEL MOD!\nhttps://www.reddit.com/r/witcher/comments/4ho4le/broke_witcher_3_w_fast_travel_mod_looking_for/'

In [98]:
len(witcher3['reviews'])

20

In [99]:
reviews_list = []

for i in range(len(witcher3['reviews'])):
    reviews_list.append(witcher3['reviews'][i]['review'])

In [102]:
reviews_list

['Gwent.',
 'In my top 3 games of all time.\nAlso in an effort to save some unfortunate soul from my mistake, I must warn you....if you mod this game and get the Fast Travel from anywhere mod, you can completely break this game and lock yourself out from being able to complete main quests.\nDO NOT USE THE FAST TRAVEL MOD!\nhttps://www.reddit.com/r/witcher/comments/4ho4le/broke_witcher_3_w_fast_travel_mod_looking_for/',
 "This is my 4th playthrough of Witcher 3, played it thrice on GOG. You can already see where I am going with this review. \nSprint! YES FINALLY! (DA:Inquisition still doesn't have it). Drink a werewolf decoction and run like the wind. Exploration is taken to another pedestal in Witcher 3. You can basically ditch your adopted daughter and explore the entire Kaer Morhen area in the first 15 mins of the game (provided you skipped the long ass cut-scene in the start). Not interested in fighting the Griffin ? No problem, go ahead and do the pan quest or the other beautifully

In [104]:
reviews_list[1]

'In my top 3 games of all time.\nAlso in an effort to save some unfortunate soul from my mistake, I must warn you....if you mod this game and get the Fast Travel from anywhere mod, you can completely break this game and lock yourself out from being able to complete main quests.\nDO NOT USE THE FAST TRAVEL MOD!\nhttps://www.reddit.com/r/witcher/comments/4ho4le/broke_witcher_3_w_fast_travel_mod_looking_for/'

In [110]:
emoji_test = reviews_list[3]

In [111]:
import emot

In [117]:
#emot.emoji(emoji_test)

In [116]:
import spacy

In [119]:
test_review = reviews_list[2]

In [121]:
test_review = test_review.lower()

In [122]:
test_review

"this is my 4th playthrough of witcher 3, played it thrice on gog. you can already see where i am going with this review. \nsprint! yes finally! (da:inquisition still doesn't have it). drink a werewolf decoction and run like the wind. exploration is taken to another pedestal in witcher 3. you can basically ditch your adopted daughter and explore the entire kaer morhen area in the first 15 mins of the game (provided you skipped the long ass cut-scene in the start). not interested in fighting the griffin ? no problem, go ahead and do the pan quest or the other beautifully written side quests in white orchard or if you get easily intrigued, explore the questions marks on the map. these smart sneakily designed question marks encourage exploration and if you get bored of enjoying the sprint mechanic, call for roach! although that dumb broken horse gets annoying after a while. but...that's not it! white orchard is just the first big  area you are introduced to after kaer morhen. novigrad and

In [127]:
nlp = spacy.load('en_core_web_sm')

In [128]:
doc = nlp(test_review)

In [143]:
# for token in doc:
#     print(token, token.lemma_)

In [133]:
from sklearn.feature_extraction.text import CountVectorizer

In [140]:
cv1 = CountVectorizer(stop_words='english')

In [141]:
review_vectorized = cv1.fit_transform(test_review)

ValueError: Iterable over raw text documents expected, string object received.

In [142]:
test_review

"this is my 4th playthrough of witcher 3, played it thrice on gog. you can already see where i am going with this review. \nsprint! yes finally! (da:inquisition still doesn't have it). drink a werewolf decoction and run like the wind. exploration is taken to another pedestal in witcher 3. you can basically ditch your adopted daughter and explore the entire kaer morhen area in the first 15 mins of the game (provided you skipped the long ass cut-scene in the start). not interested in fighting the griffin ? no problem, go ahead and do the pan quest or the other beautifully written side quests in white orchard or if you get easily intrigued, explore the questions marks on the map. these smart sneakily designed question marks encourage exploration and if you get bored of enjoying the sprint mechanic, call for roach! although that dumb broken horse gets annoying after a while. but...that's not it! white orchard is just the first big  area you are introduced to after kaer morhen. novigrad and

In [149]:
for items in reviews_list:
    items = items.lower()

In [151]:
reviews_list_vectorized = cv1.fit_transform(reviews_list)

In [152]:
reviews_list_vectorized

<20x1107 sparse matrix of type '<class 'numpy.int64'>'
	with 1380 stored elements in Compressed Sparse Row format>

In [153]:
pd.DataFrame(reviews_list_vectorized.toarray(), columns=cv1.get_feature_names()).head()

,10,12,13,15,150,20,2008,21,23,30,...,ya,yeah,year,years,yen,yennefer,yes,youre,zdrave,zhenete
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,0,0,0,1,...,0,1,0,0,0,0,1,1,0,0
3,1,1,1,0,0,1,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
len(reviews_list)

20

In [157]:
import re

# Basic lsa

analyzing just the 20 reviews pulled from withcer 3, testing lsa

In [175]:
#attempting to remove all numbers
cv2 = CountVectorizer(stop_words='english', token_pattern='[a-zA-Z]+', min_df=3)

In [176]:
reviews_vectorized2 = cv2.fit_transform(reviews_list)

In [177]:
pd.DataFrame(reviews_vectorized2.toarray(), columns=cv2.get_feature_names()).head()

,actually,beautiful,best,better,blood,buy,characters,come,complete,don,...,time,triss,use,used,wait,wine,witcher,world,worth,yes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,2,0,0,0
2,0,0,0,1,1,1,1,1,0,1,...,2,1,1,1,0,1,7,2,1,1
3,1,1,0,0,0,1,0,0,1,1,...,1,0,0,0,2,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [180]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(reviews_vectorized2)


topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["topic_1","topic_2", 'topic_3', 'topic_4', 'topic_5'],
             columns = cv2.get_feature_names())
topic_word

,actually,beautiful,best,better,blood,buy,characters,come,complete,don,...,time,triss,use,used,wait,wine,witcher,world,worth,yes
topic_1,0.071,0.041,0.040,0.037,0.038,0.049,0.173,0.051,0.043,0.103,...,0.092,0.038,0.057,0.051,0.030,0.038,0.260,0.157,0.049,0.036
topic_2,-0.073,-0.035,-0.017,-0.029,-0.015,-0.042,-0.125,-0.034,0.083,-0.012,...,-0.083,-0.015,-0.025,-0.034,-0.032,-0.015,-0.043,-0.061,-0.042,-0.015
topic_3,-0.125,-0.063,-0.056,0.045,0.054,0.030,-0.172,0.021,-0.020,-0.029,...,0.067,0.054,0.031,0.021,-0.041,0.054,0.397,-0.020,0.030,0.052
topic_4,-0.114,0.027,0.145,-0.067,0.070,-0.069,0.083,-0.003,-0.007,0.059,...,-0.125,0.070,0.141,-0.003,-0.028,0.070,-0.101,-0.113,-0.069,0.071
topic_5,0.043,0.077,-0.103,-0.012,-0.049,0.129,-0.300,-0.068,0.160,0.046,...,0.209,-0.049,-0.099,-0.068,0.258,-0.049,-0.108,-0.112,0.129,-0.053


In [181]:
lsa.explained_variance_ratio_

array([0.39705531, 0.30673345, 0.10717094, 0.07812891, 0.03634503])

In [182]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [185]:
display_topics(lsa, cv2.get_feature_names(), 30)


Topic  0
geralt, game, s, witcher, t, games, just, characters, story, world, like, love, play, know, hours, don, open, explore, think, time, quests, playing, new, fun, actually, played, use, steam, need, going

Topic  1
geralt, complete, need, s, don, blood, wine, triss, stone, gwent, yes, really, effort, enjoy, best, main, masterpiece, use, thought, better, wait, going, come, used, exploring, beautiful, right, open, worth, buy

Topic  2
just, witcher, explore, hours, gwent, played, like, playing, quests, enjoy, really, time, new, effort, wine, stone, blood, triss, main, yes, know, better, need, use, worth, buy, steam, exploring, come, used

Topic  3
game, story, love, best, use, like, characters, main, yes, triss, wine, stone, blood, effort, s, don, need, explore, masterpiece, beautiful, enjoy, thought, come, used, exploring, going, playing, complete, really, gwent

Topic  4
hours, story, masterpiece, wait, time, enjoy, complete, right, buy, worth, play, fun, game, beautiful, playing

# Basic NMF

In [215]:
nmf_model = NMF(5)
doc_topic2 = nmf_model.fit_transform(reviews_vectorized2)

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [216]:
topic_word2 = pd.DataFrame(nmf_model.components_.round(3),
             index = ["topic_1","topic_2", 'topic_3', 'topic_4', 'topic_5'],
             columns = cv2.get_feature_names())
topic_word

,abilities,able,absolute,absolutely,access,account,achieve,achievements,acting,action,...,writing,written,wrong,xbox,yakuza,yeah,year,years,yes,young
topic_1,0.006,0.015,0.007,0.004,0.003,0.009,0.006,0.007,0.018,0.009,...,0.004,0.019,0.003,0.004,0.023,0.009,0.015,0.019,0.017,0.007
topic_2,-0.004,-0.011,0.029,0.001,0.002,-0.010,-0.002,-0.003,0.010,0.030,...,-0.003,0.043,0.001,-0.009,0.114,-0.017,-0.026,-0.027,-0.031,0.029
topic_3,0.015,0.014,-0.005,0.009,0.005,0.012,0.010,0.008,-0.013,-0.021,...,0.007,-0.002,0.013,-0.003,-0.017,-0.015,0.017,-0.015,-0.025,-0.016
topic_4,-0.018,-0.036,0.004,-0.007,-0.009,-0.022,-0.017,-0.018,0.027,0.008,...,-0.004,-0.007,0.006,-0.004,0.013,0.009,-0.010,-0.002,0.016,0.007
topic_5,-0.004,0.013,-0.005,-0.006,0.003,0.026,-0.007,0.011,-0.001,0.003,...,-0.002,0.045,0.003,0.033,0.000,-0.010,0.001,0.014,-0.022,0.002


In [191]:
display_topics(nmf_model, cv2.get_feature_names(), 30)


Topic  0
game, love, story, characters, s, like, t, best, use, witcher, games, world, don, just, know, main, explore, triss, stone, blood, wine, effort, yes, thought, playing, play, beautiful, think, exploring, come

Topic  1
geralt, s, witcher, complete, open, game, world, don, need, games, really, steam, t, blood, triss, wine, stone, like, know, thought, masterpiece, wait, right, worth, buy, explore, playing, better, come, characters

Topic  2
just, witcher, game, explore, hours, s, like, quests, time, know, played, gwent, playing, t, world, play, new, geralt, really, steam, effort, main, better, worth, buy, use, stone, triss, blood, wine

Topic  3
games, t, game, fun, open, characters, world, s, play, think, actually, witcher, know, quests, time, new, story, steam, just, like, played, right, buy, worth, playing, don, love, better, thought, come

Topic  4
story, hours, game, masterpiece, wait, s, time, play, enjoy, right, buy, worth, complete, like, playing, don, beautiful, fun, act

## ok we have now run lsa and nmf on just witcher reviews, now going to pull reviews from all 15 games, to get better topics 
hopefully

In [192]:
rpg_ids

['292030',
 '413150',
 '22380',
 '1145360',
 '49520',
 '391540',
 '48700',
 '435150',
 '374320',
 '213670',
 '582010',
 '22330',
 '489830',
 '261550',
 '1235140']

In [197]:
all_reviews = []

for items in rpg_ids:
    response = requests.get('http://store.steampowered.com/appreviews/{}?json=1'.format(items))
    temp = response.json()
    for i in range(len(temp['reviews'])):
        all_reviews.append(temp['reviews'][i]['review'])
        
        


In [198]:
len(all_reviews)
# that was easier than expected - must be getting better at le coding    

285

In [240]:
#all_reviews

# LSA all reviews no preprocessing

In [200]:
cv3 = CountVectorizer(stop_words='english', token_pattern='[a-zA-Z]+', min_df=3)

In [201]:
all_vectorized = cv3.fit_transform(all_reviews)

In [202]:
pd.DataFrame(all_vectorized.toarray(), columns=cv3.get_feature_names()).head()

,abilities,able,absolute,absolutely,access,account,achieve,achievements,acting,action,...,writing,written,wrong,xbox,yakuza,yeah,year,years,yes,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
3,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(all_vectorized)


topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["topic_1","topic_2", 'topic_3', 'topic_4', 'topic_5'],
             columns = cv3.get_feature_names())
topic_word

,abilities,able,absolute,absolutely,access,account,achieve,achievements,acting,action,...,writing,written,wrong,xbox,yakuza,yeah,year,years,yes,young
topic_1,0.006,0.015,0.007,0.004,0.003,0.009,0.006,0.007,0.018,0.009,...,0.004,0.019,0.003,0.004,0.023,0.009,0.015,0.019,0.017,0.007
topic_2,-0.004,-0.011,0.029,0.001,0.002,-0.010,-0.002,-0.003,0.010,0.030,...,-0.003,0.043,0.001,-0.009,0.114,-0.017,-0.026,-0.027,-0.031,0.029
topic_3,0.015,0.014,-0.005,0.009,0.005,0.012,0.010,0.008,-0.013,-0.021,...,0.007,-0.002,0.013,-0.003,-0.017,-0.015,0.017,-0.015,-0.025,-0.016
topic_4,-0.018,-0.036,0.004,-0.007,-0.009,-0.022,-0.017,-0.018,0.027,0.008,...,-0.004,-0.007,0.006,-0.004,0.013,0.009,-0.010,-0.002,0.016,0.007
topic_5,-0.004,0.013,-0.005,-0.006,0.003,0.026,-0.007,0.011,-0.001,0.003,...,-0.002,0.045,0.003,0.033,0.000,-0.010,0.001,0.014,-0.022,0.002


In [205]:
lsa.explained_variance_ratio_

array([0.38430333, 0.0480107 , 0.0385617 , 0.03663476, 0.03515316])

In [207]:
display_topics(lsa, cv3.get_feature_names(), 30)


Topic  0
game, s, t, like, story, just, games, play, really, time, good, characters, character, world, new, don, played, great, way, hours, make, rpg, fun, start, combat, dlc, playing, gameplay, bad, feel

Topic  1
characters, attacks, enemies, grind, late, weapons, important, end, damage, job, yakuza, jrpg, items, battle, difficulty, outside, level, need, probably, specific, really, point, honestly, boss, early, battles, jobs, levels, armor, pretty

Topic  2
b, s, t, geralt, really, masterpiece, ll, m, work, getting, quests, played, fun, think, ve, lot, feel, started, options, friends, original, worth, felt, thing, world, players, haven, won, time, turn

Topic  3
b, fallout, masterpiece, new, truly, vegas, story, best, dlc, rpg, don, game, great, know, character, start, journey, hit, choices, games, graphics, hades, elements, late, m, attacks, combat, sound, design, word

Topic  4
good, hours, borderlands, shoot, average, masterpiece, guys, bad, guy, price, time, ll, sale, music, fri

# nmf all reviews no preprocessing

In [220]:
nmf_model2 = NMF(5)
doc_topic2 = nmf_model2.fit_transform(all_vectorized)

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [221]:
topic_word2 = pd.DataFrame(nmf_model2.components_.round(3),
             index = ["topic_1","topic_2", 'topic_3', 'topic_4', 'topic_5'],
             columns = cv3.get_feature_names())
topic_word

,abilities,able,absolute,absolutely,access,account,achieve,achievements,acting,action,...,writing,written,wrong,xbox,yakuza,yeah,year,years,yes,young
topic_1,0.006,0.015,0.007,0.004,0.003,0.009,0.006,0.007,0.018,0.009,...,0.004,0.019,0.003,0.004,0.023,0.009,0.015,0.019,0.017,0.007
topic_2,-0.004,-0.011,0.029,0.001,0.002,-0.010,-0.002,-0.003,0.010,0.030,...,-0.003,0.043,0.001,-0.009,0.114,-0.017,-0.026,-0.027,-0.031,0.029
topic_3,0.015,0.014,-0.005,0.009,0.005,0.012,0.010,0.008,-0.013,-0.021,...,0.007,-0.002,0.013,-0.003,-0.017,-0.015,0.017,-0.015,-0.025,-0.016
topic_4,-0.018,-0.036,0.004,-0.007,-0.009,-0.022,-0.017,-0.018,0.027,0.008,...,-0.004,-0.007,0.006,-0.004,0.013,0.009,-0.010,-0.002,0.016,0.007
topic_5,-0.004,0.013,-0.005,-0.006,0.003,0.026,-0.007,0.011,-0.001,0.003,...,-0.002,0.045,0.003,0.033,0.000,-0.010,0.001,0.014,-0.022,0.002


In [222]:
display_topics(nmf_model2, cv3.get_feature_names(), 20)


Topic  0
game, s, fallout, story, like, rpg, games, dlc, truly, new, t, great, play, know, character, gameplay, vegas, bad, best, good

Topic  1
game, story, characters, s, t, like, attacks, enemies, just, games, grind, late, really, weapons, level, important, end, job, damage, character

Topic  2
game, s, t, like, just, really, time, world, play, feel, story, lot, games, played, options, fun, friends, getting, better, playing

Topic  3
b, s, masterpiece, t, m, ll, best, work, vegas, story, fallout, game, won, new, don, played, single, developed, definition, games

Topic  4
game, good, hours, borderlands, bad, just, guys, shoot, time, average, story, s, like, t, guy, masterpiece, price, games, play, friends


In [223]:
all_vectorized

<285x813 sparse matrix of type '<class 'numpy.int64'>'
	with 5580 stored elements in Compressed Sparse Row format>

# same but with tf idf instead of regular countvectorizer

In [224]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [231]:
tfv1 = TfidfVectorizer(stop_words='english', min_df = 3, token_pattern='[a-zA-Z]+')

In [232]:
inverse_vectorized = tfv1.fit_transform(reviews_list)

In [233]:
inverse_vectorized

<20x53 sparse matrix of type '<class 'numpy.float64'>'
	with 229 stored elements in Compressed Sparse Row format>

In [234]:
pd.DataFrame(inverse_vectorized.toarray(), columns=tfv1.get_feature_names()).head()

,actually,beautiful,best,better,blood,buy,characters,come,complete,don,...,time,triss,use,used,wait,wine,witcher,world,worth,yes
0,0.000000,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.326346,0.000000,...,0.281253,0.000000,0.356252,0.000000,0.000000,0.000000,0.466904,0.000000,0.000000,0.000000
2,0.000000,0.00000,0.0,0.06126,0.072286,0.066218,0.072286,0.072286,0.000000,0.057068,...,0.114136,0.072286,0.072286,0.072286,0.000000,0.072286,0.331582,0.114136,0.066218,0.072286
3,0.140471,0.12868,0.0,0.00000,0.000000,0.128680,0.000000,0.000000,0.128680,0.110899,...,0.110899,0.000000,0.000000,0.000000,0.280943,0.000000,0.000000,0.000000,0.128680,0.000000
4,0.000000,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [235]:
new_nmf = NMF(5)
new_doc_topic = new_nmf.fit_transform(inverse_vectorized)

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [258]:
new_doc_topic

array([[0.        , 0.        , 0.        , 0.93439892, 0.        ],
       [0.20619377, 0.08788052, 0.59725348, 0.01305009, 0.        ],
       [0.05388576, 0.        , 0.31900771, 0.14321137, 0.49164357],
       [0.23254416, 0.        , 0.        , 0.        , 0.41055765],
       [0.3955105 , 0.        , 0.        , 0.        , 0.        ],
       [0.15299129, 0.09269162, 0.09087423, 0.        , 0.61586228],
       [0.07663645, 0.12173725, 0.15660762, 0.03384639, 0.58885117],
       [0.15685594, 0.07564816, 0.35631833, 0.01169489, 0.49302284],
       [0.42302223, 0.        , 0.        , 0.14578511, 0.        ],
       [0.        , 0.        , 0.04618845, 0.        , 0.39159172],
       [0.14320384, 0.        , 0.        , 0.80004403, 0.15202445],
       [0.        , 0.80892985, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.84778837, 0.        , 0.        ],
       [0.4410109 , 0.09492435, 0.04268022, 0.        , 0.0600718 ],
       [0.        , 0.80892985, 0.

In [236]:
new_topic_word = pd.DataFrame(new_nmf.components_.round(3),
             index = ["topic_1","topic_2", 'topic_3', 'topic_4', 'topic_5'],
             columns = tfv1.get_feature_names())
topic_word

,abilities,able,absolute,absolutely,access,account,achieve,achievements,acting,action,...,writing,written,wrong,xbox,yakuza,yeah,year,years,yes,young
topic_1,0.006,0.015,0.007,0.004,0.003,0.009,0.006,0.007,0.018,0.009,...,0.004,0.019,0.003,0.004,0.023,0.009,0.015,0.019,0.017,0.007
topic_2,-0.004,-0.011,0.029,0.001,0.002,-0.010,-0.002,-0.003,0.010,0.030,...,-0.003,0.043,0.001,-0.009,0.114,-0.017,-0.026,-0.027,-0.031,0.029
topic_3,0.015,0.014,-0.005,0.009,0.005,0.012,0.010,0.008,-0.013,-0.021,...,0.007,-0.002,0.013,-0.003,-0.017,-0.015,0.017,-0.015,-0.025,-0.016
topic_4,-0.018,-0.036,0.004,-0.007,-0.009,-0.022,-0.017,-0.018,0.027,0.008,...,-0.004,-0.007,0.006,-0.004,0.013,0.009,-0.010,-0.002,0.016,0.007
topic_5,-0.004,0.013,-0.005,-0.006,0.003,0.026,-0.007,0.011,-0.001,0.003,...,-0.002,0.045,0.003,0.033,0.000,-0.010,0.001,0.014,-0.022,0.002


In [237]:
display_topics(new_nmf, tfv1.get_feature_names(), 20)


Topic  0
game, play, masterpiece, beautiful, better, time, yes, played, story, main, quests, enjoy, complete, wait, use, effort, hours, thought, really, wine

Topic  1
best, games, played, game, characters, time, love, use, main, fun, effort, t, think, actually, know, quests, steam, new, exploring, going

Topic  2
witcher, open, world, game, effort, use, complete, quests, main, time, games, really, explore, just, steam, know, triss, wine, stone, blood

Topic  3
gwent, better, quests, t, s, game, just, blood, stone, triss, wine, really, thought, explore, witcher, play, characters, beautiful, going, geralt

Topic  4
geralt, s, t, story, characters, just, like, hours, love, fun, games, know, think, don, world, actually, playing, explore, game, new


# so this notebook was used to get a feel for pulling reviews and trying the different nlp learned (during first week) 
...actually I'm going to try lda too, but after this, I am going to create seperate notebooks for review pulling, and nlp. this was just a test notebook

In [ ]:
# make a loop, for token  in toekns:
#     if part of speech
    
# this is notes for if I want to also do just verbs or adjectives 


In [241]:

witcher3


{'success': 1,
 'query_summary': {'num_reviews': 20,
  'review_score': 9,
  'review_score_desc': 'Overwhelmingly Positive',
  'total_positive': 143952,
  'total_negative': 3565,
  'total_reviews': 147517},
 'reviews': [{'recommendationid': '92291984',
   'author': {'steamid': '76561198366921885',
    'num_games_owned': 108,
    'num_reviews': 13,
    'playtime_forever': 6511,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 6511,
    'last_played': 1609852044},
   'language': 'english',
   'review': 'Gwent.',
   'timestamp_created': 1621491406,
   'timestamp_updated': 1621491406,
   'voted_up': True,
   'votes_up': 148,
   'votes_funny': 49,
   'weighted_vote_score': '0.842416703701019287',
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': False},
  {'recommendationid': '92366587',
   'author': {'steamid': '76561198992556699',
    'num_games_owned': 67,
    'num_reviews': 17,
    'playtime_forever': 4211,
    '

In [247]:
tfv2 = TfidfVectorizer(stop_words='english', min_df = 3, token_pattern='[a-zA-Z]+')

In [249]:
new_nmf2 = NMF(5)

In [246]:
queries=['open world']

In [252]:
vt = tfv1.transform(queries)

In [253]:
tt = new_nmf.transform(vt)

In [254]:
# shows that the new query is closely related to topics
tt


array([[0.        , 0.        , 0.33034444, 0.        , 0.09314997]])

In [257]:
all_reviews

['Gwent.',
 'In my top 3 games of all time.\nAlso in an effort to save some unfortunate soul from my mistake, I must warn you....if you mod this game and get the Fast Travel from anywhere mod, you can completely break this game and lock yourself out from being able to complete main quests.\nDO NOT USE THE FAST TRAVEL MOD!\nhttps://www.reddit.com/r/witcher/comments/4ho4le/broke_witcher_3_w_fast_travel_mod_looking_for/',
 "This is my 4th playthrough of Witcher 3, played it thrice on GOG. You can already see where I am going with this review. \nSprint! YES FINALLY! (DA:Inquisition still doesn't have it). Drink a werewolf decoction and run like the wind. Exploration is taken to another pedestal in Witcher 3. You can basically ditch your adopted daughter and explore the entire Kaer Morhen area in the first 15 mins of the game (provided you skipped the long ass cut-scene in the start). Not interested in fighting the Griffin ? No problem, go ahead and do the pan quest or the other beautifully

In [262]:
pairwise_distances(tt,  new_doc_topic, metric='cosine').argsort()

array([[15, 12,  1,  7,  2, 17,  6,  5,  9,  3, 13, 10, 16, 14,  0, 18,
         8,  4, 11, 19]])

In [268]:
new_doc_topic

array([[0.        , 0.        , 0.        , 0.93439892, 0.        ],
       [0.20619377, 0.08788052, 0.59725348, 0.01305009, 0.        ],
       [0.05388576, 0.        , 0.31900771, 0.14321137, 0.49164357],
       [0.23254416, 0.        , 0.        , 0.        , 0.41055765],
       [0.3955105 , 0.        , 0.        , 0.        , 0.        ],
       [0.15299129, 0.09269162, 0.09087423, 0.        , 0.61586228],
       [0.07663645, 0.12173725, 0.15660762, 0.03384639, 0.58885117],
       [0.15685594, 0.07564816, 0.35631833, 0.01169489, 0.49302284],
       [0.42302223, 0.        , 0.        , 0.14578511, 0.        ],
       [0.        , 0.        , 0.04618845, 0.        , 0.39159172],
       [0.14320384, 0.        , 0.        , 0.80004403, 0.15202445],
       [0.        , 0.80892985, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.84778837, 0.        , 0.        ],
       [0.4410109 , 0.09492435, 0.04268022, 0.        , 0.0600718 ],
       [0.        , 0.80892985, 0.

In [261]:
from sklearn.metrics import pairwise_distances

In [267]:
all_reviews[39]

'great'